### Imports

In [1]:
from analyze_results import *
from getting_examples import *
from predict_activations import *
from model_utils import *
from utils import *
from reVals_vs_val import *
import json
import pprint

In [2]:
sae, model = load_sae_and_model()

/root/feature_benchmark/feat_bench/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


In [3]:
string = ['hello there my small green friend']
_, inner_acts, _ = get_sae_activations(model, sae, string)

In [4]:
f_19_acts = [inner_acts[0][i][19] for i in range(len(inner_acts[0]))]
regular_losses = get_vanilla_loss(model, sae, string)
sae_losses = get_vanilla_loss(model, sae, string, with_sae_replacement=True)
precomputed_zeros = [[[0.0] * len(l) for l in seq] for seq in inner_acts]
zeros_losses = get_recons_loss_from_predicted_values(model, sae, string, precomputed_zeros)
ablated_inner_acts = replace_feature_activation(inner_acts, 19, 0)
ablated_feature_losses = get_recons_loss_from_predicted_values(model, sae, string, ablated_inner_acts)
ablated_feature_losses[0]

[10.626654624938965,
 4.0466508865356445,
 6.337054252624512,
 8.39116096496582,
 6.708599090576172,
 4.537791728973389]

In [5]:
predicted = [5*(18-i) for i in range(len(inner_acts[0]))]

replacements = [predicted]
replaced_inner_acts = replace_sequence_feature_activation(inner_acts, 19, replacements)
replaced_sae_losses = get_recons_loss_from_predicted_values(model, sae, string, replaced_inner_acts)
replaced_sae_losses

[[9.74209976196289,
  11.601560592651367,
  8.337759017944336,
  9.623224258422852,
  8.756343841552734,
  9.638862609863281]]

In [3]:
get_experiment_losses('public_data/subset_100_run1.json', model, sae)

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]


AssertionError: IS this the problem?? 20, 17

In [ ]:
# recompute_directory_activations('6-res-jb_subset_100', 'gpt2-small-organized/6-res-jb', model, sae, recompute=False, re_sort=False, num_neg_others=20)

In [6]:
file_names, mean_pos_ratio, mean_neg_ratio, num_positives = compare_recomputed_group('6-res-jb_subset_100')

In [7]:
for file, pos, neg, num_pos in zip(file_names, mean_pos_ratio, mean_neg_ratio, num_positives):
    if pos < 0.7:
        print(file, 'pos', pos)
    if neg > 0.05:
        print(file, 'neg', neg)
    if num_pos < 10:
        print(file, 'num pos', num_pos)

2534.json pos 0.5516546720958339
9532.json pos 0.6212932628651577
15570.json pos 0.4686740678896811
20780.json pos 0.342133800284376
20780.json num pos 5
2830.json pos 0.22913556500864662
10404.json pos 0
10404.json num pos 0
8064.json pos 0.3940560582069721
4036.json neg 0.2727272727272727
19899.json pos 0
19899.json num pos 0
7739.json neg 0.16
19484.json pos 0.5232896412076664
14931.json pos 0.050575466759776914
14931.json num pos 9
22832.json pos 0.5843800777466767
9579.json pos 0.14695032468931066


In [8]:
ignore_indices = [20780, 10404, 19899, 14931]
np.random.seed(42)
indices = list(map(int, np.random.choice(24576, size=100, replace=False)))
indices = [i for i in indices if i not in ignore_indices]
print(indices)
human_indices = indices[:20]

[2350, 23251, 11000, 13627, 21896, 6332, 2534, 11263, 8913, 9532, 5128, 5157, 15162, 12092, 19697, 15570, 21983, 3230, 10322, 7428, 10949, 11451, 16697, 6750, 19453, 23607, 1346, 4730, 21772, 11817, 20963, 17386, 20351, 20820, 3096, 4634, 2830, 11212, 16183, 8064, 14882, 6448, 14583, 1103, 10075, 5625, 9760, 3594, 20189, 2004, 11264, 8685, 16693, 7476, 16596, 11799, 13133, 9658, 1202, 12694, 14524, 4036, 4679, 14073, 14372, 14610, 15500, 21032, 752, 18083, 7739, 11606, 3197, 14169, 19484, 22719, 8728, 7104, 22832, 2858, 9002, 536, 7585, 17533, 428, 19505, 6769, 2726, 13410, 22202, 3509, 9579, 681, 22138, 21380, 15824]


In [4]:
results = run_experiments(
    location='6-res-jb_subset_100',
    num_features=100, 
    test_pos=10, # Experiment with
    test_neg=10, # Experiment with
    show_pos=0, # Experiment with
    show_neg=0, # Experiment with
    neg_type='others', # Experiment with
    binary_class=False, # Experiment with
    all_tokens=True,
    show_max_token=False, # Experiment with
    num_completions=5, # Experiment with
    debug=False, 
    randomize_pos=True, 
    save_location='all_features',
    feature_ids=indices
)

# # the run_experiments function automatically saves results to results/exp_{timestamp}.json
# pprint.pprint(results)

Submitted 1 of 100 tasks. Been running for 12 seconds
Submitted 2 of 100 tasks. Been running for 22 seconds
Submitted 3 of 100 tasks. Been running for 32 seconds
Submitted 4 of 100 tasks. Been running for 42 seconds
Submitted 5 of 100 tasks. Been running for 52 seconds
Submitted 6 of 100 tasks. Been running for 62 seconds
Submitted 7 of 100 tasks. Been running for 72 seconds
Submitted 8 of 100 tasks. Been running for 82 seconds
Submitted 9 of 100 tasks. Been running for 92 seconds
Submitted 10 of 100 tasks. Been running for 102 seconds
Submitted 11 of 100 tasks. Been running for 112 seconds
Submitted 12 of 100 tasks. Been running for 122 seconds
Submitted 13 of 100 tasks. Been running for 132 seconds
Submitted 14 of 100 tasks. Been running for 143 seconds
Submitted 15 of 100 tasks. Been running for 153 seconds
Submitted 16 of 100 tasks. Been running for 163 seconds
Submitted 17 of 100 tasks. Been running for 173 seconds
CRITICAL WARNING: No valid model prediction for " prepared man eve

In [3]:
feat_id = 1

description, pos_examples, neg_examples, highest_activation = get_pos_neg_examples(feat_id, layer=6, basis='res-jb', num_pos=2, num_neg=2, neg_type='others', randomize_pos_examples=False)
print(description)

max_indices = [pos_examples[i]['max_value_token_index'] + 1 for i in range(len(pos_examples))] + [9 for i in range(len(neg_examples))]
strings = [pos_examples[i]['sentence_string'] for i in range(len(pos_examples))] + [neg_examples[i]['sentence_string'] for i in range(len(neg_examples))]

for pos in pos_examples:
    print(pos['sentence_string'])
    print(pos['values'])

for neg in neg_examples:
    print(neg['sentence_string'])
    print(neg['values'])

mentions of financial funding through grants
 published this spring, was funded through a grant from NHTSA and conducted through the
[0, 0, 0, 0, 0, 0, 0, 0, 49.51050567626953, 0.1973066926002502, 0, 0, 0, 0, 0, 0, 0]
I. will receive a three-year grant from the Massachusetts Service Alliance that will place
[0, 0, 0, 0, 0, 0, 0, 0, 48.00002670288086, 0, 0, 0, 0, 0, 0, 0, 0]
 maintenance and repair.ĊĊConstruction and extraction, and management were more preferred by Generation
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
liquid GoldâĢĿ is now harder to extract in proportion to how much better the miner
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
pre_acts, inner_acts, post_acts = get_sae_activations(model, sae, strings)

for inner_act in inner_acts:
    x = np.array(inner_act)
    print(x[:,feat_id])

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.         48.49956131  1.33336258  0.
  0.          0.          0.          0.          0.          0.        ]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.         42.76519775  0.          0.
  0.          0.          0.          0.          0.          0.        ]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [5]:
# Get model's loss on strings
regular_losses = get_vanilla_loss(model, sae, strings)
print(pretty_losses_fmt("Regular", strings, regular_losses))

# Get model's loss on strings using SAE reconstructed activations
sae_losses = get_vanilla_loss(model, sae, strings, with_sae_replacement=True)
print(pretty_losses_fmt("SAE", strings, sae_losses))

# Loss with all features ablated
precomputed_zeros = [[[0.0] * len(l) for l in seq] for seq in inner_acts]
zeros_losses = get_recons_loss_from_predicted_values(model, sae, strings, precomputed_zeros)
print(pretty_losses_fmt("Zeros", strings, zeros_losses))

Regular losses:
 published this spring, was funded through a grant from NHTSA and conducted through the (4.1803): 13.6845 6.9317 6.7337 2.1646 5.5099 5.8693 3.7500 1.0739 2.1365 0.1572 6.3267 4.8857 0.0050 2.3500 5.8504 2.4335 1.2026
I. will receive a three-year grant from the Massachusetts Service Alliance that will place (4.7467): 3.9997 6.1135 11.2789 7.5890 1.7388 7.5961 0.6220 2.4118 5.1721 1.0106 0.4074 5.7994 9.1498 5.7121 3.7558 0.5996 7.7381
 maintenance and repair.ĊĊConstruction and extraction, and management were more preferred by Generation (6.6791): 15.5910 3.6142 3.4101 3.3890 13.1244 5.9245 4.1812 0.6081 12.4337 2.8099 7.9760 3.7875 3.3171 5.7862 8.2974 7.8089 10.2664 3.6485 10.9288
liquid GoldâĢĿ is now harder to extract in proportion to how much better the miner (6.1832): 11.8105 11.7301 14.0158 7.4184 6.0638 3.9871 7.3467 6.3138 5.2734 9.7803 0.3941 4.5342 3.9772 8.5973 0.1226 4.5083 0.3920 8.0369 2.1290 7.2319
SAE losses:
 published this spring, was funded through a 

In [6]:
# Get model's loss on strings using SAE with selected feature ablated
replacements = [0 for string in strings]
ablated_inner_acts = replace_max_feature_activation(inner_acts, feat_id, max_indices, replacements)
ablated_sae_losses = get_recons_loss_from_predicted_values(model, sae, strings, ablated_inner_acts)
print(pretty_losses_fmt(f"SAE feature {feat_id} ablated", strings, ablated_sae_losses))

for inner_act in ablated_inner_acts:
    x = np.array(inner_act)
    print(x[:,feat_id])

# Print how much loss changes after ablating selected feature
difference = elementwise_difference(sae_losses, ablated_sae_losses)
rounded_difference = [[round(elem, 2) for elem in sublist] for sublist in difference]
print(rounded_difference)

SAE feature 1 ablated losses:
 published this spring, was funded through a grant from NHTSA and conducted through the (4.2348): 13.7159 5.8250 7.5830 1.0363 5.8370 7.0606 3.2471 1.2248 2.1267 0.7712 6.5461 4.4802 0.2106 2.4924 5.8819 2.7817 1.1709
I. will receive a three-year grant from the Massachusetts Service Alliance that will place (4.7978): 4.0963 5.9176 10.0126 7.4673 1.9837 7.4549 0.4112 1.9489 5.1028 1.6349 0.8448 6.5888 9.1750 6.6881 4.3267 0.9515 6.9573
 maintenance and repair.ĊĊConstruction and extraction, and management were more preferred by Generation (6.7188): 15.6880 3.7076 3.9394 3.5226 13.0648 6.0707 3.1389 2.6757 13.8095 3.2645 8.0783 3.8995 3.2487 5.9011 8.1065 6.7553 8.4807 3.4476 10.8584
liquid GoldâĢĿ is now harder to extract in proportion to how much better the miner (6.0971): 11.7411 10.5847 13.4746 8.2948 5.8304 3.8520 7.7078 5.9872 5.4499 9.9195 0.6567 4.8825 4.1477 8.8427 0.1153 4.4879 0.8287 5.6379 2.4252 7.0759
[0.         0.         0.         0.        

In [9]:
predictions = predict_activations('6-res-jb_subset_100/681.json', test_pos=5, test_neg=5, show_pos=0, show_neg=0, binary_class=False, all_tokens=True, neg_type='others', show_max_token=False, num_completions=3, debug=True, randomize_pos=False, model="gpt-3.5-turbo")

You are evaluating an english description of an autoencoder feature. The description should correspond to sentences which result in high activation. The english description of the feature is: "occurrences of the word "try""
The feature will tend to have zero activation on most tokens, but may activate highly on one or a few tokens in the sentence that correspond to the provided description.
The value of the highest activation on the dataset is 54.67. Almost all tokens in almost all sequences have an activation of zero, while a few tokens that exactly match the provided description may have higher activations (somewhere between 0 and 54.67). You must predict the activations for each token a new sentence based off of the provided description. 
You MUST respond with a list of numbers, one for each token in the sequence, and NO OTHER content.
Please predict the activation on this sentence,responding with a list of 17 numbers between 0 and 54.67.
Sentence: " See our privacy notice Could not

In [10]:
print(len(predictions[1]['tokens']))
print(len(predictions[1]['values']))
print(len(predictions[1]['prediction']))
predictions[1]['tokens']

17
19
17


[' See',
 ' our',
 ' privacy',
 ' notice',
 ' Could',
 ' not',
 ' subscribe',
 ',',
 ' try',
 ' again',
 ' later',
 ' Invalid',
 ' Email',
 'Ċ',
 'Ċ',
 'An',
 ' object']

In [11]:
with open('6-res-jb_subset_100/681.json', 'r') as file:
    data = json.load(file)
print(len(data['posActivations'][1]['values']))
print(len(data['posActivations'][1]['recomputedValues'][1:]))
print(len(data['posActivations'][1]['tokens']))
print(data['posActivations'][1]['tokens'])
data['posActivations'][1]['values']
data['posActivations'][1]['recomputedValues'][1:]

17
19
17
[' See', ' our', ' privacy', ' notice', ' Could', ' not', ' subscribe', ',', ' try', ' again', ' later', ' Invalid', ' Email', 'Ċ', 'Ċ', 'An', ' object']


[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 49.46665954589844,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [22]:
strings = []
strings.append(''.join(data['posActivations'][1]['tokens']))
print(strings)
_, inner_acts, _ = get_sae_activations(model, sae, strings)

[' See our privacy notice Could not subscribe, try again later Invalid EmailĊĊAn object']


In [23]:
len(inner_acts[0])

20

In [24]:
f_19_acts = [x[19] for x in inner_acts[0]]
print(f_19_acts)
print(len(f_19_acts))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
20


In [27]:
x = model.to_tokens(
            [strings[0]],
            truncate=True,
            prepend_bos=sae.cfg.prepend_bos,
        )

In [29]:
x

tensor([[50256,  4091,   674,  6782,  4003, 10347,   407, 12383,    11,  1949,
           757,  1568, 17665,  9570,   128,   232,   128,   232,  2025,  2134]],
       device='cuda:0')

In [41]:
for t in x[0]:
    print(model.tokenizer.decode(t))

<|endoftext|>
 See
 our
 privacy
 notice
 Could
 not
 subscribe
,
 try
 again
 later
 Invalid
 Email
�
�
�
�
An
 object


### Run an experiment

### Do analysis on loaded json_data

In [ ]:
json_data_binary = load_json_results('results/binary_test/exp_binary_others.json')
json_data_continuous = load_json_results('results/binary_test/exp_continuous_others.json')

In [ ]:
binary_preds = [json_data_binary['results'][i]['gpt_predictions'] for i in range(len(json_data_binary['results']))]
continuous_preds = [json_data_continuous['results'][i]['gpt_predictions'] for i in range(len(json_data_continuous['results']))]

In [ ]:
accuracy = get_binary_accuracy(binary_preds, plot_cdf=True, plot_distribution=True)

In [ ]:
accuracy_descs = get_accuracy_descs(json_data_binary, include_pos_neg=True, display=True)

In [ ]:
resave_organized_modeldata(autoencoder_layers = [6],
                        autoencoder_bases = [
                            'neurons',
                            'res-jb',])